# Crashes on a Street

In [1]:
import json
import pytz
import folium
from folium.features import DivIcon
from folium.plugins import HeatMap

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from matplotlib._color_data import TABLEAU_COLORS

import os
os.chdir('..')
from scripts.crash_data_analysis import CrashDataAnalysis

In [2]:
cda = CrashDataAnalysis()
most_recent_crash = cda.most_recent_crash_timestamp()

## Find ID of street

In [3]:
query = """
select
sr.gid
, sr.lrsroute
, st_AsGeoJSON(sr.geom) as street_line
, sc.fullname
, count(distinct sc.masterid) as count_of_masterids
, row_number() over (partition by sr.gid, sr.lrsroute order by count(distinct sc.masterid) desc) 
    as fullname_priority

from street_routes sr
inner join street_centerline sc using (lrsroute)

group by 1,2,3,4
"""

street_ids = pd.read_sql(query, cda.conn)

In [4]:
streets_to_map = street_ids[street_ids.lrsroute.str.contains('1ST')].reset_index()
streets_to_map

,index,gid,lrsroute,street_line,fullname,count_of_masterids,fullname_priority
0,57,57,31ST AV1,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",E 31ST AVE,21,1
1,58,58,31ST AV2,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",E 31ST AVE,6,1
2,59,59,31ST AV3,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",E 31ST AVE,10,1
3,60,60,31ST AV4,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",E 31ST AVE,1,1
4,61,61,31ST PL,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",E 31ST PL,2,1
...,...,...,...,...,...,...,...
114,5059,4949,W51ST AV6,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",W 51ST AVE,1,1
115,5060,4950,W51ST PL,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",W 51ST PL,1,1
116,6444,6319,SRACECDS1ST,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",S RACE ST,1,1
117,6640,6513,SVERBECD1ST,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",S VERBENA ST,1,1


In [5]:
this_map = folium.Map(prefer_canvas=True, tiles='CartoDB Positron')

street_color = TABLEAU_COLORS['tab:orange']

for idx, row in streets_to_map.iterrows():
    folium.GeoJson(row.street_line, style_function=lambda x: {'color': street_color}).add_to(this_map)

    street_line_dict = json.loads(row.street_line)
    middle_point = int(np.round(len(street_line_dict['coordinates'][0]) / 2))
    
    folium.map.Marker(
        [street_line_dict['coordinates'][0][middle_point][1], street_line_dict['coordinates'][0][middle_point][0]]
        , icon=DivIcon(
            icon_size=(250,36)
            , icon_anchor=(0,0)
            , html=f'<div style="font-size: 20pt; color:{street_color}">gid: {row.gid}</div>'
        )
     ).add_to(this_map)

this_map.fit_bounds(this_map.get_bounds())
this_map

## Heatmap of Crashes on One Street

In [6]:
# street_gid = 125 # E 14th
# street_gid = 3900 # Speer

# street_gid = 4689 # N Federal
street_gid = 7584 # S Federal

In [7]:
start_date = pytz.timezone('America/Denver').localize(datetime(2013,1,1))

# end_date = pytz.timezone('America/Denver').localize(datetime(2023,6,1))
end_date = pytz.timezone('America/Denver').localize(datetime.now())

In [8]:
query = f"""
select
sr.gid
, sr.lrsroute
, st_length(sr.geom_denver) / 5280 as length_miles
, c.sbi
, c.fatality
, c.pedestrian_ind
, c.bicycle_ind
, c.geo_lon
, c.geo_lat
, c.reported_date
, c.reported_date at time zone 'America/Denver' as reported_date_local
, c.incident_address_corrected
, c.incident_id

from street_routes sr
inner join crashes c on st_dwithin(sr.geom_denver, c.geom_denver, 25)

where sr.gid = {street_gid}
and reported_date at time zone 'America/Denver' > '{start_date.strftime('%Y-%m-%d')}'
and reported_date at time zone 'America/Denver' < '{end_date.strftime('%Y-%m-%d')}'
and not at_freeway

order by reported_date desc
"""

street_crashes = pd.read_sql(query, cda.conn)

In [9]:
street_crashes.groupby(street_crashes.reported_date_local.dt.year).agg(
    crashes=('reported_date_local', 'count')
    , fatalities=('fatality', 'sum')
    , sbi=('sbi', 'sum')
    , pedestrian_ind=('pedestrian_ind', 'sum')
    , bicycle_ind=('bicycle_ind', 'sum')
).astype(int)

,crashes,fatalities,sbi,pedestrian_ind,bicycle_ind
reported_date_local,,,,,
2013,378,1,7,13,8
2014,432,1,12,21,4
2015,467,2,12,13,4
2016,491,2,15,19,4
2017,456,6,7,22,4
2018,391,2,7,14,5
2019,312,0,8,17,4
2020,236,1,4,10,3
2021,214,1,9,10,2


In [10]:
this_map = folium.Map(prefer_canvas=True, tiles='CartoDB Positron')
HeatMap(street_crashes[['geo_lat', 'geo_lon']]).add_to(this_map)
this_map.fit_bounds(this_map.get_bounds())
this_map

In [11]:
days_in_data = (end_date - start_date).days

In [12]:
days_between_crashes = days_in_data / len(street_crashes)
days_between_crashes

1.0966159882295243

In [13]:
crashes_per_mile_per_week = (len(street_crashes) / street_crashes.iloc[0].length_miles) / (days_in_data/7)
crashes_per_mile_per_week

1.4595252110247643

In [14]:
street_crashes.sort_values(by='reported_date').tail(10)

,gid,lrsroute,length_miles,sbi,fatality,pedestrian_ind,bicycle_ind,geo_lon,geo_lat,reported_date,reported_date_local,incident_address_corrected,incident_id
9,7584,SFEDERAL BD1,4.373528,False,False,0.0,0.0,-105.025061,39.689484,2025-03-10 05:17:00+00:00,2025-03-09 23:17:00,W FLORIDA AVE / S FEDERAL BLVD,2025120669
8,7584,SFEDERAL BD1,4.373528,False,False,0.0,0.0,-105.025045,39.684971,2025-03-10 21:48:00+00:00,2025-03-10 15:48:00,S FEDERAL BLVD / W COLORADO AVE,2025122019
7,7584,SFEDERAL BD1,4.373528,False,False,0.0,0.0,-105.025045,39.684971,2025-03-10 21:57:00+00:00,2025-03-10 15:57:00,S FEDERAL BLVD / W COLORADO AVE,2025121994
6,7584,SFEDERAL BD1,4.373528,False,False,0.0,0.0,-105.025095,39.711172,2025-03-10 22:03:00+00:00,2025-03-10 16:03:00,S FEDERAL BLVD / W ALAMEDA AVE,2025121853
5,7584,SFEDERAL BD1,4.373528,False,False,0.0,0.0,-105.025086,39.702077,2025-03-11 21:34:00+00:00,2025-03-11 15:34:00,S FEDERAL BLVD / W OHIO AVE,2025123946
4,7584,SFEDERAL BD1,4.373528,False,False,0.0,0.0,-105.025088,39.714823,2025-03-14 14:21:00+00:00,2025-03-14 08:21:00,S FEDERAL BLVD / W BAYAUD AVE,2025114719
3,7584,SFEDERAL BD1,4.373528,False,False,0.0,0.0,-105.024992,39.701706,2025-03-17 21:16:00+00:00,2025-03-17 15:16:00,800 BLOCK S FEDERAL BLVD,2025135452
2,7584,SFEDERAL BD1,4.373528,False,False,0.0,0.0,-105.025061,39.689484,2025-03-19 01:20:00+00:00,2025-03-18 19:20:00,S FEDERAL BLVD / W FLORIDA AVE,2025137729
1,7584,SFEDERAL BD1,4.373528,False,False,0.0,0.0,-105.025039,39.682269,2025-03-22 03:37:00+00:00,2025-03-21 21:37:00,W JEWELL AVE / S FEDERAL BLVD,2025143110
0,7584,SFEDERAL BD1,4.373528,False,False,0.0,0.0,-105.024954,39.683172,2025-03-27 21:22:00+00:00,2025-03-27 15:22:00,1800 BLOCK S FEDERAL BLVD,2025153829


In [15]:
cda.conn.dispose()

In [16]:
len(street_crashes)

4078